# Document, Compile, Install, and Distribute

There is few more steps to make the tool better for users and easy to distribute. Some of these steps are required for inclusion into the grass-addons repository.

## Addons Repository

- Community-maintained tools (addons aka extensions aka plugins)
- Separate from the main repository, but only one repository
- A repository with the source code, not just a registry
- Best of both worlds:
  * Broader community of contributors (including one-time contributors)
  * Single repository maintained by the core community similarly to the main repository

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: viewshed
# % keyword: geometry
# % keyword: statistics
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# % guisection: Input
# %end
# %option G_OPT_V_INPUT
# % key: points
# % guisection: Input
# %end
# %option G_OPT_R_INPUTS
# % guisection: Input
# %end
# %option G_OPT_V_OUTPUT
# % guisection: Output
# %end
# %option G_OPT_M_NPROCS
# %end


import atexit
import multiprocessing
import csv
import io
import tempfile

from pathlib import Path

import grass.script as gs


def data_to_csv(results, file_name, csv_header):
    with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(
            csv_file, fieldnames=csv_header, extrasaction="ignore"
        )
        writer.writeheader()
        for point, result in results:
            row = {}
            for raster_name, statistics in result.items():
                for key, value in statistics.items():
                    new_key = f"{raster_name}_{key}"
                    point[new_key] = value
            row.update(point)
            writer.writerow(row)

def output_results(results, sample_rasters, output):
    with tempfile.TemporaryDirectory() as tmp_dir:
        file_name = Path(tmp_dir) / "data.txt"
        csv_header = ["cat", "east", "north"]
        database_columns = ["cat INTEGER", "east REAL", "north REAL"]
        for raster in sample_rasters:
            for value in ["mean", "stddev", "min", "max"]:
                name = f"{raster}_{value}"
                csv_header.append(name)
                database_columns.append(f"{name} REAL")
        data_to_csv(results, file_name, csv_header)
        gs.run_command(
            "v.in.ascii",
            input=file_name,
            output=output,
            format="point",
            separator="comma",
            cat=1,
            x=2,
            y=3,
            skip=1,
            columns=database_columns,
        )


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def multiple_viewsheds(
    elevation,
    points,
    sample_rasters,
    output,
    nprocs,
):
    point_data = gs.read_command(
        "v.out.ascii",
        input=points,
        type="point",
        format="point",
        separator="comma",
        flags="cr",
    )
    reader = csv.DictReader(io.StringIO(point_data))
    data = []
    for point in reader:
        data.append((elevation, point, sample_rasters))

    with multiprocessing.Pool(processes=nprocs) as pool:
        results = pool.starmap(parametrized_viewshed, data)
    output_results(results, sample_rasters, output)


def parametrized_viewshed(elevation, point, sample_rasters):
    coordinates = (point["east"], point["north"])
    result = one_viewshed(
        elevation=elevation,
        coordinates=coordinates,
        sample_rasters=sample_rasters,
    )
    return point, result


def one_viewshed(
    elevation,
    coordinates,
    sample_rasters,
):
    viewshed = gs.append_node_pid(
        gs.legalize_vector_name(f"tmp_viewshed_{coordinates}")
    )
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
        quiet=True,
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    return results


def main():
    options, unused_flags = gs.parser()
    sample_rasters = options["input"].split(",")
    multiple_viewsheds(
        elevation=options["elevation"],
        points=options["points"],
        sample_rasters=sample_rasters,
        output=options["output"],
        nprocs=int(options["nprocs"]),
    )


if __name__ == "__main__":
    main()

## Name

Tool name should follow the existing conventions for naming GRASS tools. Tools are organized into categories (families) based on their function. The categories are distinguished by prefixes:

| Prefix | Function | Example |
| --- | --- | --- |
| r | raster processing | r.mapcalc: map algebra |
| v | vector processing | v.clean: topological cleaning |
| i | imagery processing | i.segment: object recognition |
| db | database management | db.select: select values from table |
| r3 | 3D raster processing | r3.stats: 3D raster statistics |
| t | temporal data processing | t.rast.aggregate: temporal aggregation |
| g | general data management | g.rename: renames map |
| d | display | d.rast: display raster map |
| ps | PostScript rendering | ps.map: create map compositions |
| m | miscellaneous | m.proj: convert coordinates |

The name of the module helps to understand its function, for example v.in.lidar starts with v so it deals with vector maps, the name follows with in which indicates that the module is for importing the data into GRASS GIS Spatial Database and finally lidar indicates that it deals with lidar point clouds. 

Generally, the idea is to include only one or two dots. All core tools comply with this rule, but some addons break it. Sometimes, this suggests further grouping. For example, tools staring with `v.net` deal with vector network analysis and tools starting with `g.gui` are opening GUI applications.

Tools with non-compliant names will generally work, but may not make full use of some tools such as the GUI which uses the naming scheme to recognize GRASS tools in some contexts.

## Directory Structure

The tool is in a directory named the same as the tool. The Python file has the name and `.py` extension.

So far we have only a Python file, but all other files are in the directory as well. There can be subdirectories for special purposes; for example, tests are in a subdirectory.

Now, we create the directory and rename our script from the previous notebook to comply with the rules:

In [ ]:
!mkdir -p v.buffered.raster
!cp vector_to_raster.py v.buffered.raster/v.buffered.raster.py

## Keywords

Keywords are part of the basic description of the tools and its interface. The first two keywords are special. First keyword is the tool family of the tool, so, e.g., for vector tools, which has names starting with `v.`, the keyword is `vector`. Second keyword is a topic which is more highlighted in the documentation than other keywords. If possible, tools should use one of [existing topics](https://grass.osgeo.org/grass82/manuals/topics.html). Tool should have at least one other keyword. These can include other data types the tool works with, the name of the specific process it implements, or synonyms for the terms used in its name and description. Keywords can contain more than one word and can be understood as general labels or tags as long as they are adding to identification of the tool in searches. Reuse of [existing keywords](https://grass.osgeo.org/grass82/manuals/keywords.html) is encouraged. Keywords in Python are specified using as follows:

```python
# %module
# % description: Converts vector data to raster data
# % keyword: vector
# % keyword: conversion
# % keyword: raster
# % keyword: rasterization
# %end
```

## Documentation

A file with documentation which uses simple HTML syntax must be provided. This documentation is then distributed with the addon and it is also automatically available online ([GRASS GIS Addons Manual pages](https://grass.osgeo.org/grass82/manuals/addons/)). A template with the main sections follows (the syntax highlighting does not work in notebook in JupyterLab, only in a separate editor tab).

In [ ]:
%%writefile v.buffered.raster/v.buffered.raster.html
<h2>DESCRIPTION</h2>

A long description with details about the method, implementation, usage or whatever is appropriate.

<h2>NOTES</h2>

Random notes, tricks, and quirks which don't fit above.

<h2>EXAMPLES</h2>

Examples of how the tool can be used alone or in combination with other tools.
Possibly using the GRASS North Carolina State Plane Metric sample Location.
At least one screenshot (PNG format) of the result should provided to show the user what to expect.

<h2>REFERENCES</h2>

Reference or references to paper related to the tool or references which algorithm the tool is based on.

<h2>SEE ALSO</h2>

List of related or similar GRASS tools or tools used together with this tools as well as any related websites, or
related pages at the GRASS GIS User wiki.

<h2>AUTHORS</h2>

List of author(s), their organizations and funding sources.

The name of the HTML file should be the name of the tool with `.html` extension.

It is a good idea to include one ore more images to enhance the documentation. These should be PNGs, GIFs, or JPEGs, but if there are original files used to generate the images, these should be included as well (SVGs, scripts, notebooks). PNGs are preferred. GIFs are for animations. JPEGs for photographs.

The image files should be named uniquely. The best practice is to use the name of the tool, but use underscores instead of dots, and an optional suffix. A PNG named like this without any additional suffix, may be used as an image representing the tool (this is currently done for tools in the main repository). All extensions should be lowercase (e.g., `.png`). The extension recognized for JPEG is `.jpg`.

Optionally, a _README.md_ file can be included if some files or other aspects of the tool need more explanation which does not fit into any of the other files, e.g., when extra instructions are needed for re-creating the images or for maintenance of the code.

## Formal requirements

Tools included in the grass-addons repository, must be under the GNU GPL license, version 2 or later (SPDX: GPL-2.0-or-later). There is a specified way how the first comment in tool's main file should look like. Here is a template for the first lines of a file:

```python
#!/usr/bin/env python

##############################################################################
# MODULE:    vector_to_raster
#
# AUTHORS:   Alice Doe <email AT some domain>
#            Bob Doe <email AT some domain>
#
# PURPOSE:   Describe your script here from maintainer perspective
#
# COPYRIGHT: (C) 2022 Alice Doe and the GRASS Development Team
#
#            This program is free software under the GNU General Public
#            License (>=v2). Read the file COPYING that comes with GRASS
#            for details.
##############################################################################

"""Describe your script here from Python user perspective"""
```

## Compilation

Although Python is not a compiled language like C, we need to compile also the Python tools in order to include them into our GRASS installation, make them executable without the file extension, and to create HTML documentation. For this a `Makefile` needs to be written which follows a standard template as well. The included `Script.make` takes care of processing everything, given that the Python script, the HTML documentation and an optional screenshot(s) in PNG format are present in the same directory. Installed tools will show up in the GUI.

In [ ]:
%%writefile v.buffered.raster/Makefile
MODULE_TOPDIR = ../..

PGM = v.buffered.raster

include $(MODULE_TOPDIR)/include/Make/Script.make

default: script

To compile, either a low level `make` command can be used, but it is easier to make use of installation mechanism of _g.extension_ which compiles and installs on Linux and macOS (and all unix-like systems).

On Linux and macOS:

In [ ]:
!grass --tmp-location XY --exec g.extension v.buffered.raster url=v.buffered.raster

In [ ]:
!grass --tmp-location XY --exec which v.buffered.raster

The best way to get the tools to Windows users is to include them in the grass-addons repository. (Experimentally, it is also possible to setup a private institution-specific repository like the grass-addons repository.)

Code can be hosted on GitHub or other platform. _g.extension_ supports installation from many sources, but it needs the compilation tools which are not available on Windows, so this works only for Linux and macOS.

## Submitting to the GRASS GIS Addons Repository

Create a pull request to the [grass-addons repository](https://github.com/OSGeo/grass-addons/) (instructions are there). Wait for someone to review it or convince someone to do that. When issues from the review are addressed, the reviewer will merge it.

Finally, check the [Submitting Guidelines](https://trac.osgeo.org/grass/wiki/Submitting) (both for updating your files and updating the guidelines themselves).

PR reviews are time consuming, so make it easier for the reviewer by checking the best practices yourself. And yes, you can become a reviewer and get access to the grass-addons repository too. That's actually much simpler than getting write access to the main repository.

## Testing your code

One way to speed up the review process is to include tests of your tool. This not only demonstrates that the tool works, but it makes it also easier to maintain the code in the future.

### grass.gunittest tests

- Based on highly customized extension of the standard Python _unittest_ package.
- Code runs in an existing GRASS session.
- Can assume the sample NC SPM location is the current location.
- Many specialized functions for GRASS GIS, especially specialized asserts.

The readily available test real-world data and assert functions specialized for GRASS GIS, make _grass.gunittest_ a great tool for tests of data processing tools.

### pytest tests

- Use _pytest_ as is.
- There are no specialized functions for GRASS GIS yet.
- Fixtures and comparisons need to be written using basic functions.
- No GRASS session or data.

The lack of any setup may work well for tests of tools which are not doing standard processing. Increasing number of project and people migrate to _pytest_, so you may simply prefer that.

## Grouping Related Tools

### Naming

Just use names which start with the same prefix or contain the same words, but keep thinks separate.

### Common Directory

If it makes sense to use each tool individually, the tools can be in separate directories.

One directory with multiple tools in subdirectories works well even for mix of Python and C tools. There should be an additional HTML documentation in the top directory. The name of the directory should be the common prefix in the name of the tools. If there is no common prefix, the tools should likely be separate.

However, a step further, including common libraries to this structure, is more complicated and the functionality is not as stable as it should be. Creating a common library is not worth the trouble for a couple of short functions shared over couple of the tools, but it may be better for maintenance of a large library of broadly used functions. Note that the tools can also call each other possibly avoiding needs for Python imports. 

### Experimental Toolboxes

#### Addon Toolbox

Multiple tools can be listed together in an XML file and _g.extension_ can show and install this toolbox.

#### GUI Toolbox

Multiple tools can be listed together in an XML file which is stored in user configuration directory. The GUI adds these toolboxes to the tree in the _Tools_ tab.